In [69]:
import sys
sys.path.append("../")

In [70]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json

In [71]:
import Misc.env_vars as ev

In [72]:
from Models.models import od_dq_base, dq_agg_sum, dq_agg_view
from Logic.base_queries import run_stmt
import Misc.env_vars as ev

In [73]:
from Logic import base_queries as bq

In [74]:
from sqlalchemy import Select, func, or_, desc

## Top Cards 

### Total Orders with Delta

In [75]:
dt_rng = bq.get_date_range()

max_date = dt_rng[1]

In [76]:
max_date

datetime.date(2024, 6, 25)

In [77]:
prev_dt = max_date - timedelta(days=1)

In [78]:
top_cards = (
    Select(
        dq_agg_sum.c.ord_date.label("Order_Date"),
        func.sum(dq_agg_sum.c.total_orders).label("Total_Orders"),
        func.sum(dq_agg_sum.c.total_canceled_orders).label("Cancelled_Orders")
    ).where(or_(dq_agg_sum.c.ord_date == max_date, dq_agg_sum.c.ord_date == prev_dt))
    .group_by(dq_agg_sum.c.ord_date)
    .order_by(desc(dq_agg_sum.c.ord_date))
)

In [79]:
print(top_cards)

SELECT db_stage.aggregated_sum.ord_date AS "Order_Date", sum(db_stage.aggregated_sum.total_orders) AS "Total_Orders", sum(db_stage.aggregated_sum.total_canceled_orders) AS "Cancelled_Orders" 
FROM db_stage.aggregated_sum 
WHERE db_stage.aggregated_sum.ord_date = :ord_date_1 OR db_stage.aggregated_sum.ord_date = :ord_date_2 GROUP BY db_stage.aggregated_sum.ord_date ORDER BY db_stage.aggregated_sum.ord_date DESC


In [80]:
res = run_stmt(top_cards)

In [81]:
df_temp = pd.DataFrame(res)

In [82]:
# df_temp.loc[:,["Order_Date"]] = df_temp["Order_Date"].astype()
df_temp.loc[:,"Total_Orders"] = df_temp["Total_Orders"].astype(int)
df_temp.loc[:,"Cancelled_Orders"] = df_temp["Cancelled_Orders"].astype(int)

In [83]:
df_temp["Cancel_percentage"] = df_temp["Cancelled_Orders"] / df_temp["Total_Orders"]

In [84]:
df_temp["Completed_percentage"] = (df_temp["Total_Orders"] - df_temp["Cancelled_Orders"])/ df_temp["Total_Orders"]

In [85]:
df_temp

,Order_Date,Total_Orders,Cancelled_Orders,Cancel_percentage,Completed_percentage
0,2024-06-25,200731,3796,0.018911,0.981089
1,2024-06-24,209041,3582,0.017135,0.982865


In [86]:
def calc_metrices(df: pd.DataFrame, col_name: str):
    old_val = np.round(df[col_name][1],4)
    new_val = np.round(df[col_name][0],4)
    diff = new_val - old_val
    per_diff = np.round((diff/old_val)*100,4)
    return new_val, diff, per_diff

In [87]:
tt, td, tp = calc_metrices(df_temp, "Total_Orders")

In [88]:
total_orders = {
    "title":"Total Orders",
    "count": str(tt),
    "increased" : False if td < 0 else True,
    "variancePercentage" : str(tp),
    "varianceText": "vs Yesterday"
}

In [89]:
tc, cd, cp = calc_metrices(df_temp, "Cancelled_Orders")

In [90]:
total_cancellation = {
    "title":"Cancelled Orders",
    "count": str(tc),
    "increased" : False if cd < 0 else True,
    "variancePercentage" : str(cp),
    "varianceText": "vs Yesterday"
}

In [91]:
cct, ccd, ccp = calc_metrices(df_temp, "Cancel_percentage")

In [92]:
cct

np.float64(0.0189)

In [93]:
cancel_percentage = {
    "title":"Order Cancellation %",
    "count": str(cct),
    "increased" : False if ccd < 0 else True,
    "variancePercentage" : str(ccp),
    "varianceText": "vs Yesterday"
}

In [94]:
cot, cod, cop = calc_metrices(df_temp, "Completed_percentage")

In [95]:
compl_percentage = {
    "title":"Order Completion %",
    "count": str(cot),
    "increased" : False if cod < 0 else True,
    "variancePercentage" : str(cop),
    "varianceText": "vs Yesterday"
}

In [96]:
final_list = [total_orders, total_cancellation, cancel_percentage,compl_percentage]

In [97]:
final_list

[{'title': 'Total Orders',
  'count': '200731',
  'increased': False,
  'variancePercentage': '-3.9753',
  'varianceText': 'vs Yesterday'},
 {'title': 'Cancelled Orders',
  'count': '3796',
  'increased': True,
  'variancePercentage': '5.9743',
  'varianceText': 'vs Yesterday'},
 {'title': 'Order Cancellation %',
  'count': '0.0189',
  'increased': True,
  'variancePercentage': '10.5263',
  'varianceText': 'vs Yesterday'},
 {'title': 'Order Completion %',
  'count': '0.9811',
  'increased': False,
  'variancePercentage': '-0.1831',
  'varianceText': 'vs Yesterday'}]

## Missing Columns Completed

In [98]:
col_sum_comp = (
    Select(
        dq_agg_view.c.ord_date, 
        func.sum(dq_agg_view.c.total_orders).label("total_orders"),
        func.sum(dq_agg_view.c.null_fulfilment_id).label("null_fulfilment_id"),
        func.sum(dq_agg_view.c.null_net_tran_id).label("null_net_tran_id"),
        func.sum(dq_agg_view.c.null_qty.label("null_qty")),
        func.sum(dq_agg_view.c.null_itm_fulfilment_id).label("null_itm_fulfilment_id"),
        func.sum(dq_agg_view.c.null_del_pc).label("null_del_pc"),
        func.sum(dq_agg_view.c.null_created_date_time).label("null_created_date_time"),
        func.sum(dq_agg_view.c.null_domain).label("null_domain"),
        func.sum(dq_agg_view.c.null_del_cty).label("null_del_cty"),
        func.sum(dq_agg_view.c.null_ord_stats).label("null_ord_stats"),
        func.sum(dq_agg_view.c.null_fulfil_status).label("null_fulfil_status"),
        func.sum(dq_agg_view.c.null_itm_cat).label("null_itm_cat"),
        func.sum(dq_agg_view.c.null_cat_cons).label("null_cat_cons"),
        func.sum(dq_agg_view.c.null_sell_pincode).label("null_sell_pincode"),
        func.sum(dq_agg_view.c.null_prov_id).label("null_prov_id"),
        func.sum(dq_agg_view.c.null_itm_id).label("null_itm_id"),
        func.sum(dq_agg_view.c.null_sell_np).label("null_sell_np"),
        func.sum(dq_agg_view.c.null_net_ord_id).label("null_net_ord_id"),
        func.sum(dq_agg_view.c.null_sell_cty).label("null_sell_cty"),
    ).where(or_(dq_agg_view.c.ord_date==max_date, dq_agg_view.c.ord_date==prev_dt))
    .group_by(dq_agg_view.c.ord_date)
    .order_by(desc(dq_agg_view.c.ord_date))
)

In [99]:
col_sum_canc = (
    Select(
        dq_agg_view.c.ord_date, 
        func.sum(dq_agg_view.c.total_canceled_orders).label("total_canceled_orders"),
        func.sum(dq_agg_view.c.null_cans_code).label("null_cans_code"),
        func.sum(dq_agg_view.c.null_cans_dt_time).label("null_cans_dt_time")
    ).where(or_(dq_agg_view.c.ord_date==max_date, dq_agg_view.c.ord_date==prev_dt))
    .group_by(dq_agg_view.c.ord_date)
    .order_by(desc(dq_agg_view.c.ord_date))
)

In [100]:
print(col_sum_comp)

SELECT db_stage.aggregated_view.ord_date, sum(db_stage.aggregated_view.total_orders) AS total_orders, sum(db_stage.aggregated_view.null_fulfilment_id) AS null_fulfilment_id, sum(db_stage.aggregated_view.null_net_tran_id) AS null_net_tran_id, sum(db_stage.aggregated_view.null_qty) AS sum_1, sum(db_stage.aggregated_view.null_itm_fulfilment_id) AS null_itm_fulfilment_id, sum(db_stage.aggregated_view.null_del_pc) AS null_del_pc, sum(db_stage.aggregated_view.null_created_date_time) AS null_created_date_time, sum(db_stage.aggregated_view.null_domain) AS null_domain, sum(db_stage.aggregated_view.null_del_cty) AS null_del_cty, sum(db_stage.aggregated_view.null_ord_stats) AS null_ord_stats, sum(db_stage.aggregated_view.null_fulfil_status) AS null_fulfil_status, sum(db_stage.aggregated_view.null_itm_cat) AS null_itm_cat, sum(db_stage.aggregated_view.null_cat_cons) AS null_cat_cons, sum(db_stage.aggregated_view.null_sell_pincode) AS null_sell_pincode, sum(db_stage.aggregated_view.null_prov_id) AS

In [101]:
res = run_stmt(col_sum_comp)

In [102]:
res_canc = run_stmt(col_sum_canc)

In [103]:
tmp_canc = pd.DataFrame(res_canc)

In [104]:
tmp_canc

,ord_date,total_canceled_orders,null_cans_code,null_cans_dt_time
0,2024-06-25,3796,50,0
1,2024-06-24,3582,124,0


In [105]:
tmp_df = pd.DataFrame(res)

In [106]:
tmp_df

,ord_date,total_orders,null_fulfilment_id,null_net_tran_id,sum_1,null_itm_fulfilment_id,null_del_pc,null_created_date_time,null_domain,null_del_cty,null_ord_stats,null_fulfil_status,null_itm_cat,null_cat_cons,null_sell_pincode,null_prov_id,null_itm_id,null_sell_np,null_net_ord_id,null_sell_cty
0,2024-06-25,200731,0,0,0,0,0,0,0,6462,0,0,708,7513,17,0,0,0,0,17
1,2024-06-24,209041,0,0,0,1,0,0,0,4837,0,0,756,5079,19,0,0,0,0,19


In [107]:
# per_df = pd.DataFrame()

In [108]:
# per_df["ord_date"] = tmp_df["ord_date"]

In [109]:
# per_df

### Calculate Percentage and Difference 

In [110]:
curr_total = int(tmp_df["total_orders"][0])
curr_total

200731

In [111]:
cols_dict = {
    "curr_date": "Current Date",
    "ord_date": "Order Date",
    "seller_np": "Seller NP",
    "null_fulfilment_id": "Fulfilment ID",
    "null_net_tran_id": "Net Transaction ID",
    "null_qty": "Quantity",
    "null_itm_fulfilment_id": "Item Fulfilment ID",
    "null_del_pc": "Delivery Pincode",
    "null_created_date_time": "Created Date",
    "null_domain": "Domain",
    "null_del_cty": "Delivery City",
    "null_cans_code": "Cancellation Code",
    "null_cans_dt_time": "Cancellation Date",
    "null_ord_stats": "Order Status",
    "null_fulfil_status": "Fulfilment Status",
    "null_itm_cat": "Item Category",
    "null_cat_cons": "Category",
    "null_sell_pincode": "Seller Pincode",
    "null_prov_id": "Provider ID",
    "null_itm_id": "Item ID",
    "null_sell_np": "Null Seller NP",
    "null_net_ord_id": "Network Order ID",
    "null_sell_cty": "Seller City"
}

In [112]:
res = []
tmp_dict = {}

for col in tmp_df.columns:
    if col.__contains__("null"):
        per = np.round((int(tmp_df[col][0])/curr_total)*100,4)
        if per > 0:
            tmp_dict = {}
            tmp_dict['title'] = cols_dict[col]
            tmp_dict['series'] = [float(per)]
            res.append(tmp_dict)

In [113]:
res

[{'title': 'Delivery City', 'series': [3.2192]},
 {'title': 'Item Category', 'series': [0.3527]},
 {'title': 'Category', 'series': [3.7428]},
 {'title': 'Seller Pincode', 'series': [0.0085]},
 {'title': 'Seller City', 'series': [0.0085]}]

## Totals Trend

In [114]:
ord_trend = (
    Select(
        dq_agg_view.c.ord_date.label("ord_date"),
        func.sum(dq_agg_view.c.total_orders).label("total_orders"),
        func.sum(dq_agg_view.c.null_fulfilment_id).label("null_fulfilment_id"),
        func.sum(dq_agg_view.c.null_net_tran_id).label("null_net_tran_id"),
        func.sum(dq_agg_view.c.null_qty).label("null_qty"),
        func.sum(dq_agg_view.c.null_itm_fulfilment_id).label("null_itm_fulfilment_id"),
        func.sum(dq_agg_view.c.null_del_pc).label("null_del_pc"),
        func.sum(dq_agg_view.c.null_created_date_time).label("null_created_date_time"),
        func.sum(dq_agg_view.c.null_domain).label("null_domain"),
        func.sum(dq_agg_view.c.null_del_cty).label("null_del_cty"),
        func.sum(dq_agg_view.c.null_ord_stats).label("null_ord_stats"),
        func.sum(dq_agg_view.c.null_fulfil_status).label("null_fulfil_status"),
        func.sum(dq_agg_view.c.null_itm_cat).label("null_itm_cat"),
        func.sum(dq_agg_view.c.null_cat_cons).label("null_cat_cons"),
        func.sum(dq_agg_view.c.null_sell_pincode).label("null_sell_pincode"),
        func.sum(dq_agg_view.c.null_prov_id).label("null_prov_id"),
        func.sum(dq_agg_view.c.null_itm_id).label("null_itm_id"),
        func.sum(dq_agg_view.c.null_sell_np).label("null_sell_np"),
        func.sum(dq_agg_view.c.null_net_ord_id).label("null_net_ord_id"),
        func.sum(dq_agg_view.c.null_sell_cty).label("null_sell_cty"),
        func.sum(dq_agg_view.c.total_canceled_orders).label("total_canceled_orders"),
        func.sum(dq_agg_view.c.null_cans_code).label("null_cans_code"),
        func.sum(dq_agg_view.c.null_cans_dt_time).label("null_cans_dt_time") 
    ).group_by(dq_agg_view.c.ord_date)
    .order_by(desc(dq_agg_view.c.ord_date))
)

In [115]:
df_trend = pd.DataFrame(run_stmt(ord_trend))

In [116]:
df_trend.head(5)

,ord_date,total_orders,null_fulfilment_id,null_net_tran_id,null_qty,null_itm_fulfilment_id,null_del_pc,null_created_date_time,null_domain,null_del_cty,...,null_cat_cons,null_sell_pincode,null_prov_id,null_itm_id,null_sell_np,null_net_ord_id,null_sell_cty,total_canceled_orders,null_cans_code,null_cans_dt_time
0,2024-06-25,200731,0,0,0,0,0,0,0,6462,...,7513,17,0,0,0,0,17,3796,50,0
1,2024-06-24,209041,0,0,0,1,0,0,0,4837,...,5079,19,0,0,0,0,19,3582,124,0
2,2024-06-23,247323,0,0,0,0,3,0,0,6935,...,1589,33,0,0,0,0,33,6306,57,0
3,2024-06-22,187909,0,0,0,0,1,0,0,5889,...,2340,19,0,0,0,0,19,4513,62,0
4,2024-06-21,185578,0,0,0,1,1,0,0,7489,...,1760,89,0,0,0,0,89,5030,57,0


In [117]:
df_big = pd.DataFrame()
df_small = pd.DataFrame()

df_big[ev.cols_dict["ord_date"]] = df_trend[["ord_date"]]
df_small[ev.cols_dict["ord_date"]] = df_trend[["ord_date"]]

In [118]:
df_big.head(5)

,Order Date
0,2024-06-25
1,2024-06-24
2,2024-06-23
3,2024-06-22
4,2024-06-21


In [119]:
for col in df_trend.columns:
    if col not in ["ord_date", "total_orders"]:
        try:
            mean_trend = df_trend[col].mean() 
            if mean_trend > 0:
                if mean_trend > 500:
                    df_big.loc[:,[ev.cols_dict[col]]] = df_trend[col]
                elif mean_trend < 500 and mean_trend > 1:
                    df_small.loc[:,[ev.cols_dict[col]]] = df_trend[col]
                # print(col, df_trend[col].mean())
        except:
            pass

In [120]:
df_big.head(5)

,Order Date,Delivery City,Item Category,Category
0,2024-06-25,6462,708,7513
1,2024-06-24,4837,756,5079
2,2024-06-23,6935,1450,1589
3,2024-06-22,5889,882,2340
4,2024-06-21,7489,723,1760


In [121]:
# sns.lineplot(df_big)

In [122]:
df_big.to_json()

'{"Order Date":{"0":1719273600000,"1":1719187200000,"2":1719100800000,"3":1719014400000,"4":1718928000000,"5":1718841600000,"6":1718755200000,"7":1718668800000,"8":1718582400000,"9":1718496000000,"10":1718409600000,"11":1718323200000,"12":1718236800000,"13":1718150400000,"14":1718064000000,"15":1717977600000,"16":1717891200000,"17":1717804800000,"18":1717718400000,"19":1717632000000,"20":1717545600000,"21":1717459200000,"22":1717372800000,"23":1717286400000,"24":1717200000000,"25":1717113600000,"26":1717027200000,"27":1716940800000,"28":1716854400000,"29":1716768000000,"30":1716681600000,"31":1716595200000,"32":1716508800000,"33":1716422400000,"34":1716336000000,"35":1716249600000,"36":1716163200000,"37":1716076800000,"38":1715990400000,"39":1715904000000,"40":1715817600000,"41":1715731200000},"Delivery City":{"0":6462.0,"1":4837.0,"2":6935.0,"3":5889.0,"4":7489.0,"5":5467.0,"6":4558.0,"7":6279.0,"8":5296.0,"9":6787.0,"10":6700.0,"11":5672.0,"12":7856.0,"13":9127.0,"14":7849.0,"15":488

In [123]:
df_small.head(5)

,Order Date,Delivery Pincode,Seller Pincode,Provider ID,Seller City,Cancellation Code
0,2024-06-25,0,17,0,17,50
1,2024-06-24,0,19,0,19,124
2,2024-06-23,3,33,0,33,57
3,2024-06-22,1,19,0,19,62
4,2024-06-21,1,89,0,89,57


Jsonify

In [124]:
df_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order Date     42 non-null     object
 1   Delivery City  42 non-null     object
 2   Item Category  42 non-null     object
 3   Category       42 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [125]:
for col in df_big.columns:
    try:
        if col != "Order Date":
            df_big.loc[:,[col]] = df_big[col].astype(int)
    except:
        pass

In [126]:
test_dict = df_big.head(5).to_dict()

In [127]:
test_dict.keys()

dict_keys(['Order Date', 'Delivery City', 'Item Category', 'Category'])

In [128]:
res_dict = {}
output = []

In [129]:
[val for val in test_dict["Delivery City"].values()]

[6462, 4837, 6935, 5889, 7489]

In [130]:
for key in test_dict.keys():
    if key not in ("Order Date","categories"):
        print(key)
        res_dict = {}
        res_dict["name"] = key
        res_dict["data"] = [val for val in test_dict[key].values()]
        output.append(res_dict)

Delivery City
Item Category
Category


Seller with Highest Missing Data 

cancel_highest_missing_pid_data

In [131]:
stmt_curr = Select(
  dq_agg_sum.c.ord_date,
  dq_agg_sum.c.seller_np,
  func.sum(dq_agg_sum.c.missing_from_total).label("missing_val")).where(
    dq_agg_sum.c.ord_date==max_date).group_by(
      dq_agg_sum.c.ord_date).group_by(
    dq_agg_sum.c.seller_np).order_by(desc(func.sum(dq_agg_sum.c.missing_from_total)))

In [132]:
print(stmt_curr)

SELECT db_stage.aggregated_sum.ord_date, db_stage.aggregated_sum.seller_np, sum(db_stage.aggregated_sum.missing_from_total) AS missing_val 
FROM db_stage.aggregated_sum 
WHERE db_stage.aggregated_sum.ord_date = :ord_date_1 GROUP BY db_stage.aggregated_sum.ord_date, db_stage.aggregated_sum.seller_np ORDER BY sum(db_stage.aggregated_sum.missing_from_total) DESC


In [133]:
result = run_stmt(stmt_curr, 5)

In [134]:
df = pd.DataFrame(result)

In [135]:
df

,ord_date,seller_np,missing_val
0,2024-06-25,ondc.snapdeal.com,3932
1,2024-06-25,agrimart.api.agrevolution.in,2849
2,2024-06-25,biz.polestarre.in,2588
3,2024-06-25,webapi.magicpin.in/oms_partner/ondc,2289
4,2024-06-25,agg.dominos.co.in,996


In [141]:
json_frame = {
  "id":"",
  "count":"",
  "increased":"",
  "variancePercentage":"",
  "varianceText":"vs. Yesterday"
}
json_str = ""

In [139]:
for data in df.index:
  json_frame

0
1
2
3
4


In [140]:
[{'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'},
 {'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'},
 {'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'}]

[{'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'},
 {'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'},
 {'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'}]

In [ ]:
df